# COMP0005 - GROUP COURSEWORK
# Experimental Evaluation of Search Data Structures and Algorithms

The cell below defines **AbstractSearchInterface**, an interface to support basic insert/search operations; you will need to implement this three times, to realise your three search data structures of choice among: (1) *2-3 Tree*, (2) *AVL Tree*, (3) *LLRB BST*; (4) *B-Tree*; and (5) *Scapegoat Tree*. <br><br>**Do NOT modify the next cell** - use the dedicated cells further below for your implementation instead. <br>

In [71]:
# DO NOT MODIFY THIS CELL

from abc import ABC, abstractmethod  

class AbstractSearchInterface(ABC):
    '''
    Abstract class to support search/insert operations (plus underlying data structure)
    
    '''
        
    @abstractmethod
    def insertElement(self, element):     
        '''
        Insert an element in a search tree
            Parameters:
                    element: string to be inserted in the search tree (string)

            Returns:
                    "True" after successful insertion, "False" if element is already present (bool)
        '''
        
        pass 
    

    @abstractmethod
    def searchElement(self, element):
        '''
        Search for an element in a search tree
            Parameters:
                    element: string to be searched in the search tree (string)

            Returns:
                    "True" if element is found, "False" otherwise (bool)
        '''

        pass

Use the cell below to define any auxiliary data structure and python function you may need. Leave the implementation of the main API to the next code cells instead.

In [72]:
# ADD AUXILIARY DATA STRUCTURE DEFINITIONS AND HELPER CODE HERE
def printTree(node, level=0):
    if node != None:
        printTree(node.right, level + 1)
        print(' ' * 4 * level + '-> ' + str(node.key))
        printTree(node.left, level + 1)


Use the cell below to implement the requested API by means of **2-3 Tree** (if among your chosen data structure).

In [ ]:
class TwoThreeNode:
    def __init__(self, keys=None, children=None):
        self.keys = keys if keys is not None else []
        self.children = children if children is not None else []

    def isLeaf(self):
        return len(self.children) == 0


class TwoThreeTree(AbstractSearchInterface):
    def __init__(self):
        self.root = None

    def insertElement(self, element):
        if self.root is None:
            self.root = TwoThreeNode([element])
            return True

        result = self._insert(self.root, element)
        if result is not None:
            promoted, leftChild, rightChild = result
            self.root = TwoThreeNode(keys=[promoted], children=[leftChild, rightChild])
        return True

    def _insert(self, node, element):
        """
        Recursively insert 'element' into the subtree rooted at 'node'.
        If a node overflow occurs, return a tuple (promoted, leftChild, rightChild);
        otherwise, return None.
        """

        if node.isLeaf():
            node.keys.append(element)
            node.keys.sort()
            if len(node.keys) <= 2:
                return None
            else:
                return self._split(node)
        else:
            if len(node.keys) == 1:
                if element < node.keys[0]:
                    i = 0
                else:
                    i = 1
            else:
                if element < node.keys[0]:
                    i = 0
                elif element < node.keys[1]:
                    i = 1
                else:
                    i = 2

            result = self._insert(node.children[i], element)

            if result is None:
                return None
            else:
                promoted, leftChild, rightChild = result
                pos = 0
                while pos < len(node.keys) and promoted > node.keys[pos]:
                    pos += 1
                node.keys.insert(pos, promoted)
                
                node.children.pop(i)
                node.children.insert(i, leftChild)
                node.children.insert(i + 1, rightChild)
                
                if len(node.keys) <= 2:
                    return None
                else:
                    return self._split(node)

    def _split(self, node):
        """
        Splits a node that has 3 keys (and 4 children if internal).
        Returns a tuple (promoted, leftChild, rightChild) where the middle key
        is promoted to the parent.
        """

        k0, k1, k2 = node.keys

        if node.isLeaf():
            left = TwoThreeNode([k0])
            right = TwoThreeNode([k2])
        else:
            left = TwoThreeNode([k0], node.children[:2])
            right = TwoThreeNode([k2], node.children[2:])
        return (k1, left, right)

    def searchElement(self, element):
        """
        Search for element in tree iteratively.
        Returns True if found, otherwise returns False.
        """

        node = self.root
        while node is not None:
            if element in node.keys:
                return True
            if node.isLeaf():
                return False

            if len(node.keys) == 1:
                if element < node.keys[0]:
                    node = node.children[0]
                else:
                    node = node.children[1]
            else:
                if element < node.keys[0]:
                    node = node.children[0]
                elif element < node.keys[1]:
                    node = node.children[1]
                else:
                    node = node.children[2]
        return False



Use the cell below to implement the requested API by means of **AVL Tree** (if among your chosen data structure).

In [74]:
class AVLTree(AbstractSearchInterface):

    def __init__(self, key):
        self.key = key
        self.left = None
        self.right = None
        self.height = 1

    def get_height(self, node):
        if not node:
            return 0
        return node.height

    def get_balance(self, node):
        if not node:
            return 0
        return self.get_height(self.left) - self.get_height(self.right)

    def rotateLeft(self, z):
        y = z.right
        T2 = y.left

        y.left = z
        z.right = T2

        z.height = 1 + max(self.get_height(z.left), self.get_height(z.right))
        y.height = 1 + max(self.get_height(y.left), self.get_height(y.right))

        return y

    def rotateRight(self, z):
        y = z.left
        T3 = y.right

        y.right = z
        z.left = T3

        z.height = 1 + max(self.get_height(z.left), self.get_height(z.right))
        y.height = 1 + max(self.get_height(y.left), self.get_height(y.right))

        return y

    def insert(self, root, key):
        if not root:
            return AVLTree(key), True
        elif key < root.key:
            root.left, added = self.insert(root.left, key)
        elif key > root.key:
            root.right, added = self.insert(root.right, key)
        else:
            return root, False

        root.height = 1 + max(self.get_height(root.left), self.get_height(root.right))
        balance = self.get_balance(root)

        # Left rotation
        if balance > 1 and key < root.left.key:
            return self.rotateRight(root), True

        # Right rotation
        if balance < -1 and key > root.right.key:
            return self.rotateLeft(root), True

        # Left-Right rotation
        if balance > 1 and key > root.left.key:
            root.left = self.rotateLeft(root.left)
            return self.rotateRight(root), True

        # Right-Left rotation
        if balance < -1 and key < root.right.key:
            root.right = self.rotateRight(root.right)
            return self.rotateLeft(root), True

        return root, added
        
    def insertElement(self, element):
        new_tree, added = self.insert(self, element)
        self = new_tree
        return added
    

    def searchElement(self, element):     
        if self.key == element:
            return True

        if element < self.key and self.left:
            return self.left.searchElement(element)

        elif self.right:
            return self.right.searchElement(element)   
        
        return False

Use the cell below to implement the requested API by means of **LLRB BST** (if among your chosen data structure).

In [75]:
class LLRBBST(AbstractSearchInterface):
        
    def insertElement(self, element):
        inserted = False
        # ADD YOUR CODE HERE
      
        
        return inserted
    
    

    def searchElement(self, element):     
        found = False
        # ADD YOUR CODE HERE

        
        return found  

Use the cell below to implement the requested API by means of **B-Tree** (if among your chosen data structure).

In [76]:
class BTree(AbstractSearchInterface):
        
    def insertElement(self, element):
        inserted = False
        # ADD YOUR CODE HERE
      
        
        return inserted
    
    

    def searchElement(self, element):     
        found = False
        # ADD YOUR CODE HERE

        
        return found

Use the cell below to implement the requested API by means of **Scapegoat Tree** (if among your chosen data structure).

In [77]:
class ScapegoatTree(AbstractSearchInterface):
        
    def insertElement(self, element):
        inserted = False
        # ADD YOUR CODE HERE
      
        
        return inserted
    
    

    def searchElement(self, element):     
        found = False
        # ADD YOUR CODE HERE

        
        return found 

Use the cell below to implement the **synthetic data generator** needed by your experimental framework (be mindful of code readability and reusability).

In [78]:
import string
import random

class TestDataGenerator():
    '''
    A class to represent a synthetic data generator.

    ...

    Attributes
    ----------
    
    [to be defined as part of the coursework]

    Methods
    -------
    
    [to be defined as part of the coursework]

    '''
    
    #ADD YOUR CODE HERE
    
    def __init__():
        pass
    

Use the cell below to implement the requested **experimental framework** (be mindful of code readability and reusability).

In [79]:
import timeit
import matplotlib

class ExperimentalFramework():
    '''
    A class to represent an experimental framework.

    ...

    Attributes
    ----------
    
    [to be defined as part of the coursework]

    Methods
    -------
    
    [to be defined as part of the coursework]

    '''
            
    #ADD YOUR CODE HERE
    
    def __init__():
        pass
    

Use the cell below to illustrate the python code you used to **fully evaluate** your three chosen search data structures and algortihms. The code below should illustrate, for example, how you made used of the **TestDataGenerator** class to generate test data of various size and properties; how you instatiated the **ExperimentalFramework** class to  evaluate each data structure using such data, collect information about their execution time, plot results, etc. Any results you illustrate in the companion PDF report should have been generated using the code below.

In [80]:
# ADD YOUR TEST CODE HERE 



